In [1]:
import pandas as pd
import numpy as np
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [2]:
df = pd.read_csv('C:/MyFolder/03_Kaggle Data/NewyorkTimes/ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [4]:
print('열의 개수: ',len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [5]:
print(df['headline'].isnull().values.any())

False


In [6]:
headline = []
# 헤드라인의 값들을 리스트로 저장
headline.extend(list(df.headline.values)) 
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [7]:
print('총 샘플의 개수 : {}'.format(len(headline)))

총 샘플의 개수 : 1324


In [8]:
headline = [word for word in headline if word != "Unknown"]
#headline = headline[:50]
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline)))

노이즈값 제거 후 샘플의 개수 : 1214


In [9]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [10]:
def repreprocessing(raw_sentence):
    preproceseed_sentence = raw_sentence.encode("utf8").decode("ascii",'ignore')
    # 구두점 제거와 동시에 소문자화
    return ''.join(word for word in preproceseed_sentence if word not in punctuation).lower()

preprocessed_headline = [repreprocessing(x) for x in headline]
preprocessed_headline[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [12]:
sequences = list()

for sentence in preprocessed_headline:

    # 각 샘플에 대한 정수 인코딩
    encoded = tokenizer.texts_to_sequences([sentence])[0] 
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [13]:
index_to_word = {}
for key, value in tokenizer.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[80]))

빈도수 상위 582번 단어 : right


In [14]:
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 24


In [15]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [16]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [17]:
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [18]:
print(y[:3])

[ 269  371 1115]


In [19]:
y = to_categorical(y, num_classes=vocab_size)

In [20]:
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 5s - loss: 7.6253 - accuracy: 0.0278 - 5s/epoch - 19ms/step
Epoch 2/200
244/244 - 2s - loss: 7.1115 - accuracy: 0.0295 - 2s/epoch - 7ms/step
Epoch 3/200
244/244 - 2s - loss: 6.9769 - accuracy: 0.0342 - 2s/epoch - 7ms/step
Epoch 4/200
244/244 - 2s - loss: 6.8562 - accuracy: 0.0402 - 2s/epoch - 7ms/step
Epoch 5/200
244/244 - 2s - loss: 6.7184 - accuracy: 0.0437 - 2s/epoch - 7ms/step
Epoch 6/200
244/244 - 2s - loss: 6.5632 - accuracy: 0.0486 - 2s/epoch - 7ms/step
Epoch 7/200
244/244 - 2s - loss: 6.3888 - accuracy: 0.0519 - 2s/epoch - 7ms/step
Epoch 8/200
244/244 - 2s - loss: 6.1952 - accuracy: 0.0574 - 2s/epoch - 7ms/step
Epoch 9/200
244/244 - 2s - loss: 6.0069 - accuracy: 0.0623 - 2s/epoch - 7ms/step
Epoch 10/200
244/244 - 2s - loss: 5.8279 - accuracy: 0.0643 - 2s/epoch - 7ms/step
Epoch 11/200
244/244 - 2s - loss: 5.6533 - accuracy: 0.0707 - 2s/epoch - 7ms/step
Epoch 12/200
244/244 - 2s - loss: 5.4851 - accuracy: 0.0723 - 2s/epoch - 7ms/step
Epoch 13/200
244/244 - 2

Epoch 101/200
244/244 - 2s - loss: 0.7045 - accuracy: 0.8635 - 2s/epoch - 7ms/step
Epoch 102/200
244/244 - 2s - loss: 0.6892 - accuracy: 0.8661 - 2s/epoch - 7ms/step
Epoch 103/200
244/244 - 2s - loss: 0.6805 - accuracy: 0.8644 - 2s/epoch - 7ms/step
Epoch 104/200
244/244 - 2s - loss: 0.6599 - accuracy: 0.8685 - 2s/epoch - 7ms/step
Epoch 105/200
244/244 - 2s - loss: 0.6457 - accuracy: 0.8721 - 2s/epoch - 7ms/step
Epoch 106/200
244/244 - 2s - loss: 0.6314 - accuracy: 0.8789 - 2s/epoch - 7ms/step
Epoch 107/200
244/244 - 2s - loss: 0.6185 - accuracy: 0.8790 - 2s/epoch - 7ms/step
Epoch 108/200
244/244 - 2s - loss: 0.6052 - accuracy: 0.8827 - 2s/epoch - 7ms/step
Epoch 109/200
244/244 - 2s - loss: 0.5919 - accuracy: 0.8831 - 2s/epoch - 7ms/step
Epoch 110/200
244/244 - 2s - loss: 0.5802 - accuracy: 0.8841 - 2s/epoch - 7ms/step
Epoch 111/200
244/244 - 2s - loss: 0.5715 - accuracy: 0.8872 - 2s/epoch - 7ms/step
Epoch 112/200
244/244 - 2s - loss: 0.5567 - accuracy: 0.8904 - 2s/epoch - 7ms/step
Epoc

Epoch 200/200
244/244 - 2s - loss: 0.2640 - accuracy: 0.9162 - 2s/epoch - 7ms/step


In [21]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [22]:
print(sentence_generation(model, tokenizer, 'rich', 20))

rich carranza carranza as the chancellor i ultimately own everything hes hes doomed doomed doomed doomed craft need what 2018 he


In [23]:
print(sentence_generation(model, tokenizer, 'how', 10))

how a bag of texas dirt became a times tradition wrapping
